In [10]:
%pip install pymssql

StatementMeta(default, 0, 18, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 30.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pyodbc
import pymssql

# Parameters for SQL VM connection
sql_username = "azureuser"  # Replace with your SQL VM username
sql_password = "YourSecureP@ssw0rd123!"  # Replace with your SQL VM password
database_name = "IRIS_DB"
sql_vm_ip = "104.46.206.201"

StatementMeta(default, 0, 20, Finished, Available, Finished)

In [15]:
create_database_query = f"IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = '{database_name}') CREATE DATABASE {database_name};"
create_table_query = """
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Iris' AND xtype='U')
BEGIN
    CREATE TABLE Iris (
        Id INT IDENTITY(1,1) PRIMARY KEY,
        SepalLength FLOAT NOT NULL,
        SepalWidth FLOAT NOT NULL,
        PetalLength FLOAT NOT NULL,
        PetalWidth FLOAT NOT NULL,
        Species NVARCHAR(50) NOT NULL
    )
END
"""

StatementMeta(default, 0, 24, Finished, Available, Finished)

In [18]:
def execute_sql_query(sql_vm_ip, query, use_database=None):
    try:
        # Connect to SQL Server
        conn = pymssql.connect(
            server=sql_vm_ip,
            user=sql_username,
            password=sql_password,
            database=use_database
        )
        conn.autocommit(True)  # Set autocommit to True for non-transactional commands
        cursor = conn.cursor()
        print(f"Connected to {'database' if use_database else 'server'}.")

        # Execute query
        cursor.execute(query)
        conn.commit()
        print("SQL query executed successfully.")

        # Close connection
        cursor.close()
        conn.close()
        print("Connection closed.")
    except Exception as e:
        print("An error occurred:", e)


StatementMeta(default, 0, 27, Finished, Available, Finished)

In [19]:
execute_sql_query(sql_vm_ip, create_database_query)

StatementMeta(default, 0, 28, Finished, Available, Finished)

Connected to server.
SQL query executed successfully.
Connection closed.


In [20]:
execute_sql_query(sql_vm_ip, create_table_query, use_database=database_name)

StatementMeta(default, 0, 29, Finished, Available, Finished)

Connected to database.
SQL query executed successfully.
Connection closed.


In [29]:
import requests
import pandas as pd

iris_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# Download the file locally
local_file_path = "/tmp/iris.csv"
response = requests.get(iris_url)
with open(local_file_path, "w") as file:
    file.write(response.text)
print(f"Iris dataset downloaded to {local_file_path}")

columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

# Load the dataset
iris_pandas_df = pd.read_csv(local_file_path, header=None, names=columns)

print(iris_pandas_df.head())

StatementMeta(default, 0, 38, Finished, Available, Finished)

Iris dataset downloaded to /tmp/iris.csv
   sepal_length  sepal_width  petal_length  petal_width      species
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


In [31]:
iris_spark_df = spark.createDataFrame(iris_pandas_df)
iris_spark_df.show(5)

StatementMeta(default, 0, 40, Finished, Available, Finished)

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    species|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



In [32]:
jdbc_url = f"jdbc:sqlserver://{sql_vm_ip}:1433;databaseName=IRIS_DB"
properties = {
    "user": sql_username,
    "password": sql_password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

StatementMeta(default, 0, 41, Finished, Available, Finished)

In [33]:
# Write the DataFrame to the SQL table
iris_spark_df.write.jdbc(url=jdbc_url, table="Iris", mode="overwrite", properties=properties)

StatementMeta(default, 0, 42, Finished, Available, Finished)

In [34]:
def fetch_results_from_sql(sql_vm_ip, query, use_database=None):
    try:
        # Connect to SQL Server
        conn = pymssql.connect(
            server=sql_vm_ip,
            user=sql_username,
            password=sql_password,
            database=use_database
        )
        cursor = conn.cursor()
        print(f"Connected to {'database' if use_database else 'server'}.")

        # Execute query
        cursor.execute(query)
        results = cursor.fetchall()  # Fetch all results

        # Print results
        print("Query executed successfully. Results:")
        for row in results:
            print(row)

        # Close connection
        cursor.close()
        conn.close()
        print("Connection closed.")
        return results
    except Exception as e:
        print("An error occurred:", e)

# Query to fetch top 10 results
fetch_query = "SELECT TOP 10 * FROM Iris;"

# Call the function to execute and fetch results
results = fetch_results_from_sql(sql_vm_ip, fetch_query, use_database=database_name)

StatementMeta(default, 0, 43, Finished, Available, Finished)

Connected to database.
Query executed successfully. Results:
(5.0, 2.3, 3.3, 1.0, 'Iris-versicolor')
(5.6, 2.7, 4.2, 1.3, 'Iris-versicolor')
(5.7, 3.0, 4.2, 1.2, 'Iris-versicolor')
(5.7, 2.9, 4.2, 1.3, 'Iris-versicolor')
(6.2, 2.9, 4.3, 1.3, 'Iris-versicolor')
(5.1, 2.5, 3.0, 1.1, 'Iris-versicolor')
(5.7, 2.8, 4.1, 1.3, 'Iris-versicolor')
(6.3, 3.3, 6.0, 2.5, 'Iris-virginica')
(5.8, 2.7, 5.1, 1.9, 'Iris-virginica')
(7.1, 3.0, 5.9, 2.1, 'Iris-virginica')
Connection closed.
